In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from tqdm import tqdm     
from PIL import Image

c:\Users\shash\OneDrive\Desktop\practicing_DS\venv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
ds = load_dataset("Bingsu/Cat_and_Dog")

c:\Users\shash\OneDrive\Desktop\practicing_DS\venv2\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shash\.cache\huggingface\hub\datasets--Bingsu--Cat_and_Dog. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 2000/2000 [00:00<00:00, 35024.33 examples/s]


In [4]:
class ImageDataset(Dataset):
    def __init__(self, data, compose, type):
        super().__init__()
        self.data = data
        self.compose = compose
        self.type = type

    def __len__(self):
        return len(self.data[self.type])  # Fix dataset length

    def __getitem__(self, index):
        image = self.data[self.type][index]['image']
        label = torch.tensor(self.data[self.type][index]['labels'])
        image = self.compose(image)

        return image, label

In [5]:
compose = T.Compose([
    T.Resize((64, 64)),  # Randomly crop the image to 224x224 pixels
    T.RandomHorizontalFlip(p=0.5),  # Randomly flip the image horizontally with a probability of 0.5
    T.RandomVerticalFlip(p=0.5),  # Randomly flip the image vertically with a probability of 0.5
    T.RandomRotation(degrees=30),  # Randomly rotate the image by up to 30 degrees
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
    T.ToTensor(),  # Convert the image to a PyTorch tensor
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,stride = stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.LeakyReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
    def forward(self,x):
        identity = self.shortcut(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        
        return self.relu(out)


    def __init__(self, in_channels, out_channels, stride = 1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels,kernel_size = 3, stride=stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.act = nn.GELU()
        
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size = 3, stride = 1, padding=1, bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)



In [7]:
class SimpleResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleResNet, self).__init__()

        # Initial Convolution Layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.GELU()

        # ResNet Blocks
        self.res_block1 = ResNetBlock(64, 128, stride=2)  # Downsample
        self.res_block2 = ResNetBlock(128, 256, stride=2) # Downsample
        self.res_block3 = ResNetBlock(256, 512, stride=2) # Downsample
        self.res_block4 = ResNetBlock(512,512, stride=2)

        #remove when low perf

        # Global Average Pooling and Fully Connected Layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Outputs (batch, 512, 1, 1)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)

        x = self.avgpool(x)  
        x = torch.flatten(x, 1)  # Flatten for FC layer
        x = self.fc(x)
        return x

In [8]:
model  = SimpleResNet(num_classes=1)

In [9]:
train_dataset = ImageDataset(ds, compose, 'train')
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

test_dataset = ImageDataset(ds, compose, 'test')
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=1)

In [10]:
image, label = train_dataset[0]
image.shape

torch.Size([3, 64, 64])

In [11]:
@torch.no_grad()
def evaluate_model(test):
	model.eval()
	print("Running Validation")
	correct = 0
	for X,y in tqdm(test):
		X = X.to(device)
		y = y.view(-1, 1).float().to(device)
		out = torch.sigmoid(model(X))
		prediction = (out >= 0.5).int()
		correct += prediction.item() == y.item()
	accuracy = (correct/len(test))*100
	print(f"Accuracy : {accuracy}")
	return accuracy

In [12]:
loss_fn = nn.BCEWithLogitsLoss()  # Change to BCEWithLogitsLoss
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

c:\Users\shash\OneDrive\Desktop\practicing_DS\venv2\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [13]:
max_acc = 0

In [16]:

EPOCHS = 50
model.load_state_dict(torch.load('modelBest.pth'))
model.to(device)
for epoch in range(EPOCHS):
    sum_loss = 0
    for X, y in tqdm(train_dataloader):
        model.train()
        X = X.to(device)
        y = y.view(-1, 1).float().to(device)  # Ensure correct shape

        # Forward
        out = model(X)
        loss = loss_fn(out, y)
        
        sum_loss = sum_loss + loss.item()
        # Backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=5.0)
        optimizer.step()
    avg = sum_loss/len(train_dataloader)
    scheduler.step(avg)
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {avg:.4f}")
    acc = evaluate_model(test_dataloader)
    if(acc>max_acc):
        torch.save(model.state_dict(), 'model.pth')
        max_acc = acc

    


C:\Users\shash\AppData\Local\Temp\ipykernel_24476\1225171856.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('modelBest.pth'))
100%|████

Epoch [1/50], Loss: 0.2295
Running Validation


100%|██████████| 2000/2000 [00:10<00:00, 192.66it/s]


Accuracy : 86.1


100%|██████████| 500/500 [00:28<00:00, 17.83it/s]


...

Accuracy : 88.55


100%|██████████| 500/500 [00:27<00:00, 17.87it/s]


Epoch [50/50], Loss: 0.0852
Running Validation


100%|██████████| 2000/2000 [00:10<00:00, 192.13it/s]

Accuracy : 90.05


In [15]:
comp = T.Compose([
    T.Resize((64, 64)),
    T.ToTensor(),  # Convert the image to a PyTorch tensor
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [16]:
img = comp(Image.open('doge.jpg').convert('RGB'))
img.shape

torch.Size([3, 64, 64])

In [17]:
model.eval()
out = model(img.unsqueeze(0).to(device))
out = F.sigmoid(out)
out

tensor([[0.9170]], device='cuda:0', grad_fn=<SigmoidBackward0>)